# Pipeline para el rechazo de artefactos en Electroencefalografía
En este utilizaremos la base de datos que se utiliza de ejemplo en eeglab.

## Cargamos las librerias y los archivos necesarios

In [8]:
import mne

# Cargar datos desde un archivo (suponiendo que eeglab_raw ya existe)
eeglab_raw = mne.io.read_raw_eeglab('./data/eeglab_data.set', preload=True)
#mostrar información del archivo
eeglab_raw.info
eeglab_raw.plot()

Reading /Users/jorgerivera/Library/CloudStorage/Dropbox/Programming/eeg-project/eeg-project/data/eeglab_data.fdt
Reading 0 ... 30503  =      0.000 ...   238.305 secs...
Using qt as 2D backend.
Using pyopengl with version 3.1.6


# Filtrado de la señal
Se utiliza un filtro pasa banda de 1 a 40 Hz

In [5]:
l_freq = 1.0
h_freq = 40.0
eeglab_raw_filtered =eeglab_raw.copy().filter(l_freq, h_freq, fir_design='firwin')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 423 samples (3.305 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    0.0s finished


### Acá se definen los valores típicos para ser rechazados. 
$150\mu V$ para poner un umbral de rechazo.
$1\mu V$ para identificar los canales planos 

In [6]:
reject_criteria = dict(eeg=150e-6)  # valor umbral para los canales EEG (150 µV)
flat_criteria = dict(eeg=1e-6) 

In [22]:
events=mne.events_from_annotations(eeglab_raw_filtered)
print(type(events[0]))


Used Annotations descriptions: ['rt', 'square']
<class 'numpy.ndarray'>


### Creación de eventos y épocas
Se van a crear las epocas a partir de un evento particular.

In [24]:
event_id=1
tmin, tmax = -0.2, 0.5
epochs = mne.Epochs(eeglab_raw_filtered, events[0], event_id, tmin, tmax,  reject=reject_criteria, flat=flat_criteria, preload=True)

Not setting metadata
74 matching events found
Setting baseline interval to [-0.203125, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 74 events and 91 original time points ...
    Rejecting  epoch based on EEG : ['Fz', 'FC2']
    Rejecting  epoch based on EEG : ['Cz', 'CP1', 'Pz']
    Rejecting  epoch based on EEG : ['FPz', 'EOG1', 'F3', 'Fz']
3 bad epochs dropped


In [25]:
print(type(epochs))

<class 'mne.epochs.Epochs'>


In [26]:
epochs.plot()

Using pyopengl with version 3.1.6


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
